## Diagnostics for Spliner (3d) with RQE correction

### Configure environment

In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/sa_diagnostics/spliner_rqe")
print(os.getcwd())

import numpy
numpy.random.seed(1)

In [ ]:
import storm_analysis.diagnostics.spliner.settings as settings
import storm_analysis.diagnostics.spliner.configure as configure
import storm_analysis.diagnostics.spliner.make_data as makeData
import storm_analysis.diagnostics.spliner.analyze_data as analyzeData
import storm_analysis.diagnostics.spliner.collate as collate


In [ ]:
settings.photons = [[20, 290]]

settings.iterations = 20
settings.n_frames = 20
settings.peak_locations = None
settings.smooth_psf = False
settings.test_z_range = 0.0
settings.threshold = 8.0

rqe_correction_test = True

In [ ]:
# sCMOS calibration files.
period = 19
rqe_term = 0.92

offset = numpy.zeros((settings.y_size, settings.x_size)) + settings.camera_offset
variance = numpy.ones((settings.y_size, settings.x_size)) * settings.camera_variance


rqe = numpy.ones((settings.y_size, settings.x_size))
t1 = (1.0 + (1 - rqe_term)*numpy.sin(2.0*numpy.pi*numpy.arange(rqe.shape[1])/period))

for i in range(rqe.shape[0]):
    rqe[i,:] = rqe[i,:] * t1

gain = 2.0*numpy.ones((settings.y_size, settings.x_size))/rqe

cal_file = "calib.npy"
numpy.save(cal_file, [offset, variance, gain, rqe, 2])

### Configure

In [ ]:
configure.configure(False, cal_file = cal_file)

### Make Data

In [ ]:
makeData.makeDataRQE(cal_file = cal_file)

### Analyze data

In [ ]:
# Use constant RQE in calibration file.
if not rqe_correction_test:
    rqe = numpy.ones((settings.y_size, settings.x_size))

    numpy.save(cal_file, [offset, variance, gain, rqe, 2])

In [ ]:
%time analyzeData.analyzeData()

### Collate data

In [ ]:
collate.collateRQE()

### Reference results